<a href="https://colab.research.google.com/github/franfram/Transformers-nlp/blob/main/nlp_fine_tuning_hyperparam_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip install wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.9 MB/s eta 0:00:00


In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
%env WANDB_PROJECT=fill-mask-neufitech

env: WANDB_PROJECT=fill-mask-neufitech


### Fetch text data

In [5]:

!git clone https://github.com/franfram/Transformers-nlp.git

!mkdir ./data

!cp ./Transformers-nlp/books/Libros-en-txt/* data/ 

!rm -r ./Transformers-nlp

Cloning into 'Transformers-nlp'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 168 (delta 44), reused 110 (delta 4), pack-reused 0
Receiving objects: 100% (168/168), 2.65 MiB | 2.26 MiB/s, done.
Resolving deltas: 100% (44/44), done.


### Merge data into a single file

In [6]:
import glob
import os

def merge_text_data():
  for f in glob.glob("./data/*.txt"):
        os.system("cat "+f+" >> ./data/full_text_data.txt")

In [7]:
merge_text_data()

### Load data and split into train/test

In [8]:
from datasets import load_dataset


def load_and_split_data(data_path):
  temp = load_dataset("text", data_files = data_path)["train"]
  dataset = temp.train_test_split(test_size = 0.2, shuffle=False)
  return dataset 


In [9]:
data_path = "./data/full_text_data.txt"

dataset = load_and_split_data(data_path)

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-49a8aecb1a0b3012/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 24649
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6163
    })
})

In [10]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "CenIA/distillbert-base-spanish-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [11]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")


'>>> DistilBERT number of parameters: 67M'


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
import torch

text = "Quiero comer un [MASK]."

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_n_tokens = torch.topk(mask_token_logits, 10, dim=1).indices[0].tolist()

for token in top_n_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> Quiero comer un helado.'
'>>> Quiero comer un filete.'
'>>> Quiero comer un huevo.'
'>>> Quiero comer un pollo.'
'>>> Quiero comer un pastel.'
'>>> Quiero comer un plato.'
'>>> Quiero comer un cerdo.'
'>>> Quiero comer un poco.'
'>>> Quiero comer un pavo.'
'>>> Quiero comer un conejo.'


In [14]:
sample = dataset["train"].shuffle(seed=42).select(range(7))

for row in sample:
    print(f"\n'>>> Text: {row['text']}'")


'>>> Text: Junto con  tocarlos, se durmieron todos, para que '

'>>> Text: Luego, cuando  hubieron descansado, Dorothy '

'>>> Text:     Así fueron descendiendo hasta que, finalmente, sintieron allá '

'>>> Text: que habían tenido en el bosque y hablaban todos casi al mismo tiempo. Estas buenas gentes estaban '

'>>> Text: —dijo entonce s el muñeco.  '

'>>> Text: pies en aquel retirado paraje: se ve en él, se exalta y cree divisar '

'>>> Text: -¡Menos al fantasma, menos al fantasma! -gritaron l os gemelos, continuando sus '


# Concatenate all the examples and then split the whole corpus into chunks of equal size.

## Tokenize corpus

In [15]:
def tokenize_function(data):
  result = tokenizer(data["text"])
  if tokenizer.is_fast:
    result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result



In [16]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True, # enable fast multithreading
    remove_columns=["text"]
)
tokenized_datasets


Map:   0%|          | 0/24649 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1804 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/6163 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 24649
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 6163
    })
})

In [17]:
tokenizer.model_max_length

512

In [18]:
chunk_size = 128 # memory available in google colab's GPUs

In [19]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
  print(f"'>>> Review {idx} length: {len(sample)}")

'>>> Review 0 length: 2
'>>> Review 1 length: 14
'>>> Review 2 length: 2


## Concatenate all the samples

In [20]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}

total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 18'


In [21]:
tokenized_samples.keys()

dict_keys(['input_ids', 'attention_mask', 'word_ids'])

In [22]:
concatenated_examples.items()

dict_items([('input_ids', [4, 5, 4, 1005, 1004, 1004, 1008, 1318, 1102, 1683, 14781, 20208, 1326, 1008, 1079, 5, 4, 5]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), ('word_ids', [None, None, None, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 3, 4, None, None, None])])

In [23]:
chunks = {
    k: [t[i: i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
  print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 18'


In [24]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [25]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/24649 [00:00<?, ? examples/s]

Map:   0%|          | 0/6163 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3524
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 619
    })
})

In [26]:
tokenizer.decode(lm_datasets["train"][10]["input_ids"])

'[SEP] [CLS] llenaremos con ellas la panza de esta condenada bes tia, aprovechando que duerme. " las [SEP] [CLS] siete cabritas corrieron en busca de piedras y las fueron metiendo en la barriga, hasta [SEP] [CLS] que ya no cupieron más. la madre cosió la piel con tanta presteza y suavidad, que la [SEP] [CLS] fiera no se dio cuenta de nada ni hizo el menor mov imiento. [SEP] [CLS] [SEP] [CLS] terminada ya su siesta, el lobo se levantó, y, como los guijarros que le llenaban el [SEP] [CLS] estómago le diesen mucha sed, encaminóse a un pozo para beber'

Add mask 

In [27]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [28]:
data_collator # note that vocab size is now 31k instead of ~50k? check later, may be related to the fact that CenIA is not there anymore

DataCollatorForLanguageModeling(tokenizer=DistilBertTokenizerFast(name_or_path='CenIA/distillbert-base-spanish-uncased', vocab_size=31002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [29]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
  _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
  print(f"\n'>>> {tokenizer.decode(chunk)}")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] [SEP] [CLS] w cóc [MASK]. son [MASK]entosinfantiles. com [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] el lobo y las 7 cabritas [SEP] [CLS] autor : hermanos grimm [SEP] [CLS] [SEP] [CLS] érase una vez una vieja [MASK] que [MASK] siete cabri tas, a las que quería tan tiernamente [SEP] [CLS] como 03 madre puede querer a sus hijos. un día qui [MASK] salir al bosque a buscar comida [SEP] [CLS] y llamó a sus [MASK] [MASK]. " [MASK] mías, " les dijo, " [MASK] voy al bosque ; mucho ojo [MASK] el [SEP] [CLS] lobo, pues si entra en la casa os devorará a todas sin dejar ni un

'>>> pelo. el muy bribón [SEP] [CLS] suele disfrazarse, pero lo conoceréis enseguida por su bronca voz y sus negras patas. " [SEP] [CLS] las [MASK]ritas respondieron : [MASK] tendremos mucho cuidado, madrecita [MASK] podéis marcharos [SEP] [CLS] tranquila. " despidió [MASK] la vieja con un balido y [MASK] co nfiada, emprendió su camino [MASK] [SEP] [CLS] [SEP] [CLS] no [MASK] transcurrido mucho tiempo cuando llamaron a l

Make data collator for whole word masking

In [30]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [31]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [SEP] [CLS] www. soncuentosinfantiles. com [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] el [MASK] y las 7 cabritas [SEP] [CLS] autor [MASK] hermanos grimm [SEP] [CLS] [SEP] [CLS] [MASK] [MASK] una [MASK] una vieja cabra que tenía [MASK] cabri tas [MASK] a [MASK] que quería tan tiernamente [SEP] [CLS] como una madre puede querer [MASK] sus hijos. un día qui so salir [MASK] bosque a buscar [MASK] [SEP] [CLS] y llamó a [MASK] pequeñuelas. " [MASK] [MASK], " [MASK] dijo, " [MASK] voy al bosque ; [MASK] ojo con el [SEP] [CLS] [MASK], pues si [MASK] en [MASK] casa os devorará a todas sin [MASK] ni un'

'>>> pelo. el muy bribón [SEP] [CLS] suele disfrazarse, pero [MASK] conoceréis enseguida [MASK] su bronca voz y sus negras patas. [MASK] [SEP] [CLS] [MASK] cabritas respondieron : " tendremos mucho cuidado, madrecita [MASK] podéis marcharos [SEP] [CLS] [MASK] [MASK] " despidióse la vieja con [MASK] balido y, co nfiada, emprendió su camino [MASK] [SEP] [CLS] [SEP] [CLS] [MASK] había transcur

Downsample for faster training

In [32]:

downsampled_dataset = lm_datasets # no downsampling due to small dataset

# train_size = 2000
# test_size = int(0.1 * train_size)

# downsampled_dataset = lm_datasets["train"].train_test_split(
#     train_size=train_size, test_size=test_size, seed=42
# )
# downsampled_dataset

In [33]:
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3524
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 619
    })
})

In [34]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [35]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-spanish-corpus",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True, # boost in speed
    logging_steps=logging_steps,
    report_to="wandb", 
    run_name="finetuning-basic"
)

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/franfram/distillbert-base-spanish-uncased-finetuned-spanish-corpus into local empty directory.


Download file pytorch_model.bin:   0%|          | 32.0k/257M [00:00<?, ?B/s]

Download file runs/Feb12_14-04-00_7f1995c14488/1676210852.8757823/events.out.tfevents.1676210852.7f1995c14488.…

Download file runs/Feb12_14-04-00_7f1995c14488/1676210980.2517197/events.out.tfevents.1676210980.7f1995c14488.…

Download file runs/Feb12_14-04-00_7f1995c14488/events.out.tfevents.1676210922.7f1995c14488.604.2: 100%|#######…

...

Clean file runs/Feb12_14-04-00_7f1995c14488/events.out.tfevents.1676210922.7f1995c14488.604.2:  18%|#7        …

Download file runs/Feb12_14-04-00_7f1995c14488/events.out.tfevents.1676211052.7f1995c14488.604.4: 100%|#######…

Clean file runs/Feb12_14-04-00_7f1995c14488/events.out.tfevents.1676211052.7f1995c14488.604.4: 100%|##########…

Clean file pytorch_model.bin:   0%|          | 1.00k/257M [00:00<?, ?B/s]

Using cuda_amp half precision backend


Check perplexity (exp(CrossEntropyLoss)) before model training

In [37]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 619
  Batch size = 64


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: franfram. Use `wandb login --relogin` to force relogin


>>> Perplexity: 229.05


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3524
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 168
  Number of trainable parameters = 66961434


Epoch,Training Loss,Validation Loss
1,4.048700,3.596504
2,3.539900,3.331786
3,3.431900,3.303436


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 619
  Batch size = 64


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 619
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 619
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=168, training_loss=3.664050567717779, metrics={'train_runtime': 56.999, 'train_samples_per_second': 185.477, 'train_steps_per_second': 2.947, 'total_flos': 350363032823808.0, 'train_loss': 3.664050567717779, 'epoch': 3.0})

In [38]:
wandb.finish()

NameError: ignored

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 619
  Batch size = 64


>>> Perplexity: 26.69


Push to hub

In [ ]:
trainer.push_to_hub()

Using Accelerate to remove randomness in masking process

In [ ]:
# TO DO

### Using the fine-tuned model

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="franfram/distillbert-base-spanish-uncased-finetuned-spanish-corpus"
)

In [ ]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

# Metrics WIP

In [ ]:
!pip install evaluate

In [ ]:
!python -c "import evaluate; print(evaluate.load('exact_match').compute(references=['hello'], predictions=['hello']))"

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
accuracy.description
accuracy.features

# Hyperparameter tuning

In [42]:
!pip install "ray[tune]" hyperopt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

In [48]:
from transformers import AutoModelForMaskedLM

# model_checkpoint = "CenIA/distillbert-base-spanish-uncased"
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)


def model_init(): 
  return AutoModelForMaskedLM.from_pretrained('CenIA/distillbert-base-spanish-uncased', return_dict=True)

In [49]:
?Trainer

In [88]:
from transformers import Trainer

trainer = Trainer(
    # model=model,
    model_init = model_init, 
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # push_to_hub=True, # New, test if working
)

TypeError: ignored

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3524
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 168
  Number of trainable parameters = 66961434
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [69]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
best_trial = trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", # test without ray
    n_trials=10, # number of trials
    
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    # search_alg=HyperOptSearch(metric="objective", mode="max"),
    # # Choose among schedulers:
    # # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    # scheduler=ASHAScheduler(metric="objective", mode="max"),

    raise_on_failed_trial=False, 
)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
(pid=43492) 2023-02-27 17:29:34.255003: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=43492) 2023-02-27 17:29:34.255086: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=43492) 2023-02-27 17:29:34.255098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

== Status ==
Current time: 2023-02-27 17:29:29 (running for 00:00:00.16)
Memory usage on this node: 7.6/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.38 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/_objective_2023-02-27_17-29-29
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+-------------------+-----------------+--------------------+------------------------+---------+
| Trial name             | status   | loc               |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |
|                        |          |                   |                 |                    |                ch_size |         |
|------------------------+----------+-------------------+-----------------+--------------------+------------------------+---------|
| _objective_4a757_00000 | RUNNING  | 172.28.0.12:43492 |     4.32521e-06 |                  1 |                     16 | 11.6132

2023-02-27 17:29:40,164	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-02-27 17:29:40,895	ERROR trial_runner.py:1062 -- Trial _objective_4a757_00000: Error processing event.
ray.exceptions.RayTaskError(TypeError): ray::ImplicitFunc.train() (pid=43492, ip=172.28.0.12, repr=_objective)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint
    return self._trainable_func(
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable/function_trainable.py", line 654, in _trainable_func
    output = fn()
  File "/usr/local/lib/python3.8/dist-packages/transformers/integrations.py", line 332, in dynamic_modules_import_

Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
_objective_4a757_00000,2023-02-27_17-29-38,3459ae05bfab48b6b34def4372a08c15,966576da383a,172.28.0.12,43492,1677518978,4a757_00000


2023-02-27 17:29:40,997	ERROR tune.py:794 -- Trials did not complete: [_objective_4a757_00000, _objective_4a757_00001, _objective_4a757_00002, _objective_4a757_00003, _objective_4a757_00004, _objective_4a757_00005, _objective_4a757_00006, _objective_4a757_00007, _objective_4a757_00008, _objective_4a757_00009]
2023-02-27 17:29:41,002	INFO tune.py:798 -- Total run time: 11.27 seconds (11.18 seconds for the tuning loop).
2023-02-27 17:29:41,006	WARNING tune.py:804 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
2023-02-27 17:29:41,013	WARNING experiment_analysis.py:621 -- Could not find best trial. Did you pass the correct `metric` parameter?


== Status ==
Current time: 2023-02-27 17:29:40 (running for 00:00:11.17)
Memory usage on this node: 8.3/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.38 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/_objective_2023-02-27_17-29-29
Number of trials: 10/10 (1 ERROR, 9 PENDING)
+------------------------+----------+-------------------+-----------------+--------------------+------------------------+---------+
| Trial name             | status   | loc               |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |
|                        |          |                   |                 |                    |                ch_size |         |
|------------------------+----------+-------------------+-----------------+--------------------+------------------------+---------|
| _objective_4a757_00001 | PENDING  |                   |     7.14851e-06 |                  4 |                     16 | 14.7502 |
| _

AttributeError: ignored

In [70]:
best_trial

BestRun(run_id='82345_00007', objective=4.566604137420654, hyperparameters={'learning_rate': 2.5318525210031856e-06, 'num_train_epochs': 1, 'seed': 13.259883261976741, 'per_device_train_batch_size': 32})

In [66]:
# reproduce the best training

for n, v in best_trial.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

TypeError: ignored

In [96]:
seed = '123'

for k, v in best_trial.hyperparameters.items(): 
  if k != "seed": # remove seed te avoid error
    setattr(trainer.args, k, v)
    

# for k, v in best_trial.hyperparameters.items(): 
#   if k == "seed": # remove seed te avoid error
#     setattr(trainer.args, k, np.random.seed(seed))


print(trainer.args)

TypeError: ignored

In [90]:
for n, v in best_trial.hyperparameters.items():
  print(n, v)


# learning_rate 2.5318525210031856e-06
# num_train_epochs 1
# seed 13.259883261976741
# per_device_train_batch_size 32

learning_rate 2.5318525210031856e-06
num_train_epochs 1
seed 13.259883261976741
per_device_train_batch_size 32


In [77]:
for n,v in best_trial.hyperparameters.items(): 
  setattr(trainer.args, n, v)

In [94]:
trainer.train()

TypeError: ignored